In [230]:
from __future__ import division
import nltk
import copy
import re
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import brown
from nltk import bigrams, ngrams, trigrams 
from nltk.probability import FreqDist as FreqDist  
from numpy.polynomial.polynomial import polyfit
from collections import Counter
from string import digits
from scipy import stats
import operator
from collections import OrderedDict

In [231]:
from nltk.corpus import treebank,brown

corpus = brown.tagged_sents(tagset='universal')[:-100] 

tag_dict={}
word_dict={}

for sent in corpus:
    for elem in sent:
        w = elem[0]
        tag= elem[1]

        if w not in word_dict:
            word_dict[w]=0

        if tag not in tag_dict:
            tag_dict[tag]=0

        word_dict[w]+=1
        tag_dict[tag]+=1
unigram_prob=dict()
V=len(word_dict)
for word in word_dict.keys():
    unigram_prob[word]=word_dict[word]/V
test_data= brown.tagged_sents(tagset='universal')[-100:]


## Build the start, emission and transition probability matrices for the HMM

In [232]:
start={}
transition={}
emission={}
context={}
for line in corpus:
    previous='<s>'
    for word,tag in line:
        if(previous=='<s>'):                           # First tag of the sentence
            if tag not in context:
                context[tag]=0
            context[tag]+=1
            if tag not in start:
                start[tag]=0.0
            start[tag]+=1
            previous=tag
        else:                                          # Rest of the word-tag pairs
            if tag not in context:
                context[tag]=0
            context[tag]+=1
            if previous not in transition:
                transition[previous]={}
            if tag not in transition[previous]:
                transition[previous][tag]=0
            transition[previous][tag]+=1
            previous=tag
        if tag not in emission:
            emission[tag]={}
        if word not in emission[tag]:
            emission[tag][word]=0
        emission[tag][word]+=1
    if '</s>' not in transition[previous]:
        transition[previous]['</s>']=0
    transition[previous]['</s>']+=1
emission_raw=copy.deepcopy(emission)
for tag in transition:                                 # Normalizing the counts into probability scores
    total_count=float(sum(transition[tag].values())) 
    for next_tag in transition[tag]:
        transition[tag][next_tag]/=(total_count)
for tag in start:                                      # Normalizing the counts into probability scores
    start[tag]/=float(len(corpus))

for tag in emission:                                   # Normalizing the counts into probability scores
    total_count=float(sum(emission[tag].values())) 
    for word in emission[tag]:
        emission[tag][word]/=(total_count)        

In [238]:
sentence='Hello how do you do'

#def transition(s1,s2):
    
def smoothed_emission(tag, word,delta):
    count_pair=emission_raw[tag].get(word,0)
    prob=(count_pair+delta)/(context[tag]+delta*V)
    return prob

def log(x):
    if (x==0 ):
        return -np.inf
    else:
        return np.log(x)
def pos_tagger(sentence):
    viterbi_scores=dict()
    backpointer=dict()
    N=len(context.keys())   # Number of states except accept state '</s>'
    #T=len(sentence.split())
    T=len(sentence)
    words=['dummy']
    #words.extend(sentence.split())
    words.extend(sentence)
    for s in context.keys():                                      # Initialization
        viterbi_scores[s,1]=log((start.get(s,0))*smoothed_emission(s,words[1],0.001))
        backpointer[s,1]=0
    for t in range(2,T+1):                                        # Recursion
        for s in context.keys():
            max_prob=-np.inf
            for s1 in context.keys():
                prob=viterbi_scores[s1,t-1]
                prob+=log(transition[s1].get(s,0))
                prob+=log(smoothed_emission(s,words[t],0.001))
                if(prob>=max_prob):
                    viterbi_scores[s,t]=prob
                    backpointer[s,t]=s1
                    max_prob=prob
            print(t,words[t],s,backpointer[s,t],max_prob)
    max_prob=-np.inf
    for s in context.keys():                                      # Termination                                  
        prob=(viterbi_scores[s,T])+log(transition[s].get('</s>',0))
        if prob>=max_prob:
            max_prob=prob
            viterbi_scores['</s>',T+1]=prob
            backpointer['</s>',T+1]=s      
    tag_sequence=list()
    tag='</s>'
    tag_sequence.append(tag)
    for t in range(T+1,0,-1):
        tag=backpointer[tag,t]
        tag_sequence.append(tag)
    tag_sequence.reverse()
    tag_sequence=tag_sequence[1:len(tag_sequence)-1]
    return tag_sequence

In [239]:
context.keys()

dict_keys(['DET', 'NOUN', 'ADJ', 'VERB', 'ADP', '.', 'ADV', 'CONJ', 'PRT', 'PRON', 'NUM', 'X'])

In [240]:
test_sentences=[[word[0] for word in sentence] for sentence in test_data]
test_tags=[[word[1] for word in sentence] for sentence in test_data]

In [241]:
predicted_tags=[pos_tagger(sentence) for sentence in test_sentences]

2 asked DET PRON -26.872358515481313
2 asked NOUN PRON -28.25171390558034
2 asked ADJ PRON -26.992416688864083
2 asked VERB PRON -10.617309164079312
2 asked ADP PRON -25.774793628796587
2 asked . PRON -25.17116019682638
2 asked ADV PRON -24.857846098882796
2 asked CONJ PRON -26.023713575985248
2 asked PRT PRON -25.046360430765738
2 asked PRON PRON -26.611964833548612
2 asked NUM PRON -27.54515789949499
2 asked X X -29.65219270675017
3 Quasimodo DET VERB -31.165633291476237
3 Quasimodo NOUN VERB -24.775839300860163
3 Quasimodo ADJ VERB -31.71459337324733
3 Quasimodo VERB VERB -31.33039816446515
3 Quasimodo ADP VERB -31.183162056681745
3 Quasimodo . VERB -31.943379058759277
3 Quasimodo ADV VERB -30.73170044816012
3 Quasimodo CONJ VERB -32.31579009426755
3 Quasimodo PRT VERB -30.552102094695258
3 Quasimodo PRON VERB -31.231129799595788
3 Quasimodo NUM VERB -31.848005665744
3 Quasimodo X VERB -33.40426641049775
4 recently DET NOUN -47.67655907546262
4 recently NOUN NOUN -46.110070587070375

38 typical NOUN DET -250.84489419717062
38 typical ADJ DET -239.57937001000622
38 typical VERB DET -252.70414350265304
38 typical ADP DET -254.43384391571266
38 typical . DET -254.11459767068965
38 typical ADV DET -252.83367414923293
38 typical CONJ DET -255.74905680389944
38 typical PRT DET -254.36381568898605
38 typical PRON DET -253.2719999598232
38 typical NUM DET -252.09085053206724
38 typical X DET -251.68741140370403
39 tongue-in-cheek DET ADJ -263.4544626947565
39 tongue-in-cheek NOUN ADJ -259.43858780199963
39 tongue-in-cheek ADJ ADJ -253.7790199253225
39 tongue-in-cheek VERB ADJ -262.648659270602
39 tongue-in-cheek ADP ADJ -260.7938471790698
39 tongue-in-cheek . ADJ -260.68651557364694
39 tongue-in-cheek ADV ADJ -262.06476727980265
39 tongue-in-cheek CONJ ADJ -260.31633436674804
39 tongue-in-cheek PRT ADJ -260.738229648754
39 tongue-in-cheek PRON ADJ -262.8629019903818
39 tongue-in-cheek NUM ADJ -261.0605099052413
39 tongue-in-cheek X ADJ -261.36898655021434
40 way DET ADJ -2

4 which . ADP -43.031964021428806
4 which ADV ADP -41.60456819020655
4 which CONJ ADP -43.32085066512979
4 which PRT ADP -41.05599800201466
4 which PRON ADP -39.96879175401549
4 which NUM ADP -39.6126157030442
4 which X ADP -34.560471624476925
5 I DET DET -47.8992561864566
5 I NOUN DET -35.41860473020334
5 I ADJ DET -43.70537950738729
5 I VERB DET -45.7952472071047
5 I ADP DET -47.52494762016428
5 I . DET -47.20570137514131
5 I ADV DET -45.924777853684574
5 I CONJ DET -48.840160508351076
5 I PRT DET -47.45491939343768
5 I PRON DET -30.91248725219614
5 I NUM DET -45.18195423651888
5 I X DET -37.869760328840464
6 address DET PRON -53.68907654233688
6 address NOUN PRON -43.94115424250344
6 address ADJ PRON -53.80913471571965
6 address VERB PRON -41.42814807923573
6 address ADP PRON -52.591511655652155
6 address . PRON -51.987878223681946
6 address ADV PRON -51.67456412573836
6 address CONJ PRON -52.84043160284081
6 address PRT PRON -51.863078457621306
6 address PRON PRON -53.4286828604041

20 particular NOUN DET -145.47700585171788
20 particular ADJ DET -141.70852505916963
20 particular VERB DET -155.85364832861922
20 particular ADP DET -157.58334874167883
20 particular . DET -157.26410249665582
20 particular ADV DET -155.9831789751991
20 particular CONJ DET -158.8985616298656
20 particular PRT DET -157.51332051495223
20 particular PRON DET -156.42150478578938
20 particular NUM DET -155.2403553580334
20 particular X DET -154.8369162296702
21 branch DET ADJ -165.58361774391994
21 branch NOUN ADJ -151.4410717481127
21 branch ADJ ADJ -162.81692975380113
21 branch VERB ADJ -157.17641198518172
21 branch ADP ADJ -162.92300222823323
21 branch . ADJ -162.81567062281033
21 branch ADV ADJ -164.19392232896607
21 branch CONJ ADJ -162.44548941591145
21 branch PRT ADJ -162.86738469791743
21 branch PRON ADJ -164.99205703954522
21 branch NUM ADJ -163.18966495440472
21 branch X ADJ -163.49814159937773
22 of DET NOUN -174.34179152271517
22 of NOUN NOUN -172.77530303432292
22 of ADJ NOUN -

3 latter . DET -33.56876397074832
3 latter ADV DET -32.28784044929159
3 latter CONJ DET -35.20322310395808
3 latter PRT DET -33.81798198904469
3 latter PRON DET -32.726166259881865
3 latter NUM DET -31.545016832125885
3 latter X DET -31.1415777037627
4 obviously DET ADJ -42.32624029848172
4 obviously NOUN ADJ -38.31036540572482
4 obviously ADJ ADJ -39.55955230836291
4 obviously VERB ADJ -41.52043687432722
4 obviously ADP ADJ -39.66562478279498
4 obviously . ADJ -39.55829317737213
4 obviously ADV ADJ -29.57443068045281
4 obviously CONJ ADJ -39.188111970473216
4 obviously PRT ADJ -39.61000725247918
4 obviously PRON ADJ -41.734679594106964
4 obviously NUM ADJ -39.932287508966496
4 obviously X ADJ -40.240764153939516
5 require DET ADV -50.91719800221642
5 require NOUN ADV -52.42265427960784
5 require ADJ ADV -49.80990826247002
5 require VERB ADV -38.68358137259588
5 require ADP ADV -50.315083163345186
5 require . ADV -50.15423334231711
5 require ADV ADV -49.75269985242002
5 require CONJ AD

...

14 sprue PRT ADP -132.1154387288384
14 sprue PRON ADP -131.02823248083925
14 sprue NUM ADP -130.67205642986795
14 sprue X ADP -132.5286671306159
15 , DET NOUN -147.43017876744372
15 , NOUN NOUN -145.86369027905147
15 , ADJ NOUN -147.12239605673275
15 , VERB NOUN -145.39078959143194
15 , ADP NOUN -144.72756148756312
15 , . NOUN -126.7191811366509
15 , ADV NOUN -146.0088592515244
15 , CONJ NOUN -144.80457220463308
15 , PRT NOUN -145.76697064326248
15 , PRON NOUN -146.16436800287676
15 , NUM NOUN -145.86556446692794
15 , X NOUN -139.80426088014454
16 yaws DET . -148.1456876302641
16 yaws NOUN . -148.64176810173367
16 yaws ADJ . -148.50437980405488
16 yaws VERB . -148.31059420372281
16 yaws ADP . -148.23910529738924
16 yaws . . -147.75976341441566
16 yaws ADV . -147.69661032911608
16 yaws CONJ . -146.84482801693022
16 yaws PRT . -147.92882164586246
16 yaws PRON . -147.5049505147389
16 yaws NUM . -147.65316415905562
16 yaws X . -147.56820047890022
17 , DET CONJ -167.46833799784199
17 , NOUN

16 the PRT ADP -121.04569689612268
16 the PRON ADP -119.95849064812352
16 the NUM ADP -119.60231459715222
16 the X ADP -113.4522244524598
17 next DET DET -125.01436938729832
17 next NOUN DET -121.05111110246396
17 next ADJ DET -108.24630728093283
17 next VERB DET -122.91036040794643
17 next ADP DET -114.12136060271784
17 next . DET -124.32081457598304
17 next ADV DET -112.60574583959581
17 next CONJ DET -125.95527370919281
17 next PRT DET -124.57003259427941
17 next PRON DET -123.47821686511658
17 next NUM DET -122.2970674373606
17 next X DET -121.89362830899742
18 time DET ADJ -132.12139996568314
18 time NOUN ADJ -113.85046944887127
18 time ADJ ADJ -129.35471197556433
18 time VERB ADJ -124.40684176221342
18 time ADP ADJ -129.46078444999642
18 time . ADJ -129.35345284457355
18 time ADV ADJ -130.73170455072926
18 time CONJ ADJ -128.98327163767465
18 time PRT ADJ -129.4051669196806
18 time PRON ADJ -131.52983926130838
18 time NUM ADJ -129.7274471761679
18 time X ADJ -130.03592382114093
1

8 shut ADP VERB -66.88057984274636
8 shut . VERB -67.6407968448239
8 shut ADV VERB -66.42911823422473
8 shut CONJ VERB -68.01320788033216
8 shut PRT VERB -66.24951988075988
8 shut PRON VERB -66.92854758566041
8 shut NUM VERB -67.54542345180862
8 shut X VERB -69.10168419656236
9 , DET VERB -76.88417243801183
9 , NOUN VERB -78.09578078197949
9 , ADJ VERB -77.43313251978293
9 , VERB VERB -77.04893731100074
9 , ADP VERB -76.90170120321734
9 , . VERB -59.7835021367449
9 , ADV VERB -76.45023959469572
9 , CONJ VERB -78.03432924080315
9 , PRT VERB -76.27064124123086
9 , PRON VERB -76.94966894613138
9 , NUM VERB -77.5665448122796
9 , X VERB -72.21405077771811
10 and DET . -81.21000863035809
10 and NOUN . -81.70608910182766
10 and ADJ . -81.56870080414888
10 and VERB . -81.3749152038168
10 and ADP . -81.30342629748323
10 and . . -80.82408441450968
10 and ADV . -80.76093132921008
10 and CONJ . -62.76676955078319
10 and PRT . -80.99314264595648
10 and PRON . -80.5692715148329
10 and NUM . -80.7174

23 Cherokee ADP DET -180.9414686945817
23 Cherokee . DET -180.6222224495587
23 Cherokee ADV DET -179.34129892810196
23 Cherokee CONJ DET -182.25668158276846
23 Cherokee PRT DET -180.87144046785505
23 Cherokee PRON DET -179.77962473869223
23 Cherokee NUM DET -178.59847531093627
23 Cherokee X DET -178.19503618257306
24 strain DET ADJ -194.08823848723975
24 strain NOUN ADJ -179.79727800335016
24 strain ADJ ADJ -191.32155049712094
24 strain VERB ADJ -186.37368028377003
24 strain ADP ADJ -191.42762297155303
24 strain . ADJ -191.32029136613016
24 strain ADV ADJ -192.69854307228587
24 strain CONJ ADJ -190.95011015923126
24 strain PRT ADJ -191.3720054412372
24 strain PRON ADJ -193.49667778286502
24 strain NUM ADJ -191.69428569772452
24 strain X ADJ -192.00276234269754
25 in DET NOUN -202.69799777795262
25 in NOUN NOUN -194.22275451024512
25 in ADJ NOUN -202.39021506724166
25 in VERB NOUN -200.65860860194084
25 in ADP NOUN -183.23427368347686
25 in . NOUN -199.86541621570979
25 in ADV NOUN -201

21 dress ADP X -173.85396942149103
21 dress . X -172.2390067742112
21 dress ADV X -175.00205451522115
21 dress CONJ X -173.37770826930856
21 dress PRT X -174.26330354907464
21 dress PRON X -174.87015686607384
21 dress NUM X -175.87283895831763
21 dress X X -166.9710532831409
22 was DET VERB -185.17615550737122
22 was NOUN NOUN -184.94585209311668
22 was ADJ VERB -185.7251155891423
22 was VERB NOUN -168.3795582646782
22 was ADP NOUN -183.80972330162837
22 was . NOUN -183.6797590192661
22 was ADV VERB -184.74222266405508
22 was CONJ NOUN -183.88673401869832
22 was PRT VERB -184.56262431059025
22 was PRON VERB -185.24165201549076
22 was NUM NOUN -184.94772628099318
22 was X X -181.80857281080077
23 stupefying DET VERB -188.92788239207513
23 stupefying NOUN VERB -190.13949073604277
23 stupefying ADJ VERB -189.4768424738462
23 stupefying VERB VERB -189.09264726506402
23 stupefying ADP VERB -188.94541115728063
23 stupefying . VERB -189.70562815935818
23 stupefying ADV VERB -188.493949548759


In [242]:
def find_accuracy(predicted_tags, test_tags):
    num_sentences=len(predicted_tags)
    scores=list()
    for i in range(num_sentences):
        count=0
        predicted=predicted_tags[i]
        test=test_tags[i]
        len_sentence=len(test)
        for j in range(len_sentence):
            if(predicted[j]==test[j]):
                count+=1
            #else:
                #print(predicted[j],test[j])
        if(count/len_sentence<0.5):
            print(test_sentences[i])
            print(predicted)
            print(test)
            print("\n")
        scores.append(count/len_sentence)
    return np.mean(np.asarray(scores))
    #return scores

In [243]:
find_accuracy(predicted_tags,test_tags)

0.9103542356640717

In [244]:
lis=['It', 'is', ',', 'I', 'am', 'reliably']#, 'given', 'to', 'understand', ',', 'the', 'technical', 'argot', 'for', 'those', 'who', 'engage', 'in', 'your', 'particular', 'branch', 'of', 'the', 'boost']
pos_tagger(lis)

2 is DET PRON -27.796476779748456
2 is NOUN PRON -29.17583216984748
2 is ADJ PRON -27.916534953131226
2 is VERB PRON -8.271560885645185
2 is ADP PRON -26.69891189306373
2 is . PRON -26.095278461093525
2 is ADV PRON -25.78196436314994
2 is CONJ PRON -26.947831840252388
2 is PRT PRON -25.970478695032877
2 is PRON PRON -27.536083097815755
2 is NUM PRON -28.46927616376213
2 is X X -29.65219270675017
3 , DET VERB -28.81988501304211
3 , NOUN VERB -30.031493357009765
3 , ADJ VERB -29.368845094813203
3 , VERB VERB -28.984649886031022
3 , ADP VERB -28.837413778247615
3 , . VERB -11.71921471177518
3 , ADV VERB -28.38595216972599
3 , CONJ VERB -29.970041815833422
3 , PRT VERB -28.20635381626113
3 , PRON VERB -28.885381521161662
3 , NUM VERB -29.502257387309875
3 , X VERB -24.1497633527484
4 I DET . -33.14572120538837
4 I NOUN . -25.12440850543903
4 I ADJ . -33.50441337917916
4 I VERB . -33.310627778847085
4 I ADP . -33.23913887251351
4 I . . -32.75979698953996
4 I ADV . -32.69664390424037
4 I CON

['PRON', 'VERB', '.', 'PRON', 'VERB', '.']

In [171]:
start

{'DET': 0.21339972047519218,
 '.': 0.08901118099231307,
 'PRON': 0.15971348707197763,
 'NOUN': 0.14129979035639412,
 'ADV': 0.09117749825296995,
 'ADP': 0.12283368273934311,
 'VERB': 0.045090845562543676,
 'NUM': 0.016788958770090845,
 'CONJ': 0.04916142557651992,
 'ADJ': 0.0343466107617051,
 'PRT': 0.03665269042627533,
 'X': 0.0005241090146750524}

In [209]:
emission['X']

{'ad': 0.004382761139517896,
 'hoc': 0.002191380569758948,
 'de': 0.03287070854638422,
 'facto': 0.005113221329437546,
 'coup': 0.0014609203798392988,
 "d'etat": 0.0014609203798392988,
 'et': 0.021183345507669833,
 'al': 0.005843681519357195,
 'en': 0.003652300949598247,
 'route': 0.0029218407596785976,
 'La': 0.017531044558071585,
 'Dolce': 0.003652300949598247,
 'Vita': 0.003652300949598247,
 'De': 0.015339663988312637,
 'Al': 0.0007304601899196494,
 'Cavaliere': 0.0007304601899196494,
 'avant': 0.0007304601899196494,
 'garde': 0.0007304601899196494,
 'Le': 0.003652300949598247,
 'Theatre': 0.0007304601899196494,
 "D'Art": 0.0007304601899196494,
 'Du': 0.005843681519357195,
 'Ballet': 0.002191380569758948,
 'Espagnol': 0.0007304601899196494,
 'Sinfonica': 0.0007304601899196494,
 'Siciliana': 0.0007304601899196494,
 'Orchestre': 0.0007304601899196494,
 'Philharmonique': 0.0007304601899196494,
 'corps': 0.005113221329437546,
 'ballet': 0.0007304601899196494,
 'Capello': 0.0007304601899

In [210]:
emission['X']['am']

KeyError: 'am'

In [211]:
context.keys()

dict_keys(['DET', 'NOUN', 'ADJ', 'VERB', 'ADP', '.', 'ADV', 'CONJ', 'PRT', 'PRON', 'NUM', 'X'])

In [215]:
log(transition['X']['X'])

-0.6679024212626378

In [216]:
log(transition['PRON']['VERB'])

-0.34755962723375744

In [234]:
log(smoothed_emission('X','am',0.001))

0


-14.169617106397261

In [235]:
log(smoothed_emission('VERB','am',0.001))

222


-6.711472309434258

In [236]:
emission_raw['VERB']

{'said': 1942,
 'produced': 90,
 'took': 424,
 'had': 5093,
 'deserves': 16,
 'was': 9756,
 'conducted': 55,
 'been': 2467,
 'charged': 57,
 'investigate': 11,
 'won': 68,
 'received': 163,
 'did': 993,
 'find': 392,
 'are': 4329,
 'recommended': 46,
 'act': 74,
 'have': 3889,
 'studied': 79,
 'revised': 14,
 'modernizing': 3,
 'improving': 16,
 'commented': 18,
 'purchasing': 13,
 'operated': 27,
 'follow': 90,
 'accepted': 94,
 'inure': 2,
 'proposed': 82,
 'believes': 43,
 'should': 865,
 'be': 6334,
 'combined': 39,
 'achieve': 51,
 'reduce': 62,
 'Purchasing': 2,
 'is': 9997,
 'lacking': 26,
 'experienced': 52,
 'urged': 34,
 'take': 572,
 'remedy': 3,
 'provide': 214,
 'enabling': 13,
 're-set': 1,
 'may': 1289,
 'effected': 12,
 'granted': 53,
 'has': 2424,
 'seen': 275,
 'distribute': 6,
 'receives': 20,
 'realize': 69,
 'might': 657,
 'disable': 1,
 'feel': 201,
 'receive': 76,
 'do': 1255,
 'will': 2097,
 'continue': 104,
 'place': 53,
 'protected': 31,
 'found': 534,
 'incor